In [1]:
#importe de librerias
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gzip
import json



variables importantes para procesasar: genres, tags, specs, id

In [3]:
#cargar el dataset
datasets=pd.read_json("../datasets/data comprimida/steam_games.json.gz", compression='gzip', lines=True)

In [4]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [5]:
dt2=datasets[['release_date','price','developer']]

In [ ]:
dt2

In [6]:
datasets.drop('title',axis=1,inplace=True)

In [7]:
#elimine los nulos
datasets.dropna(how='all', inplace=True)
datasets.dropna(subset=['genres','tags','specs','id','app_name','release_date','price','developer'], inplace=True)
datasets.head(3)

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com


reindexear, tomar las columnas importantes para el modelo

In [8]:
#reset del index
datasets.reset_index(drop=True,inplace=True)

In [9]:
#paso los nombre a minusculas
datasets['app_name'] = datasets['app_name'].str.lower()

In [12]:
#Guardo del dataset mas limpoio
#datasets.to_json('../datasets/datos.json')
datasets.to_csv('../datasets/datos.csv',index=False)

In [11]:
df = pd.read_json('../datasets/datos.json')

In [10]:
generos=df['genres'].astype(str)
tags=df['tags'].astype(str)
specs=df['specs'].astype(str)


In [11]:
vec=TfidfVectorizer()
vec_matrix1= vec.fit_transform(generos)
vec_matrix2= vec.fit_transform(tags)
vec_matrix3= vec.fit_transform(specs)

In [12]:
matrix_completa=np.column_stack([vec_matrix1.toarray(),vec_matrix2.toarray(),vec_matrix3.toarray()])

In [13]:
coseno=cosine_similarity(matrix_completa)

In [ ]:
coseno

In [ ]:
datasets.head(2)

In [23]:
id_del_juego=248820.0
juego_en_data= datasets[datasets['id']== id_del_juego]

In [34]:
if not juego_en_data.empty:
    juego_indice=juego_en_data.index[0]
    juegos_similares= coseno[juego_indice]
    juegos_mas_similares=np.argsort(-juegos_similares)
    top_5_juegos=datasets.loc[juegos_mas_similares[0:6],'app_name']
    top_5_juegos_mostrar=top_5_juegos.to_numpy().tolist()
    nombre_del_juego= top_5_juegos_mostrar.pop(0)
    print(f'los 5 juegos recomendados para el id {id_del_juego} ({nombre_del_juego}) son:' )
    print(top_5_juegos_mostrar)
else:
    print('Juego el juego no esta en la base de datos')

los 5 juegos recomendados para el id 248820.0 (risk of rain) son:
['neurovoider', 'death road to canada', 'warlocks vs shadows', 'dead pixels', 'crawl']
